# Blackstone Companies

In [225]:
#URl https://doppler.finra.org/doppler-lookup/api/v1/search/firms?hl=true&nrows=99000&query=Blackstone&r=2500&wt=json
import urllib.request
from bs4 import BeautifulSoup
import json
import numpy as np
import pandas as pd

In [226]:
def getdata(search=''):
    firm_name=[]
    source_id=[]
    url="https://doppler.finra.org/doppler-lookup/api/v1/search/firms?hl=true&nrows=99000&query="+search+"&r=2500&wt=json"
    page = urllib.request.urlopen(url)
    data=json.loads(page.read())
    data=data['results']['BROKER_CHECK_FIRM']['results']
    if len(data)==0:
        return pd.DataFrame(columns=['bc_firm_name','bc_source_id'])
    else:
        for i in range(len(data)):
            datax=data[i]['fields']
            if(datax['score']>0.4):
                firm_name.append(datax['bc_firm_name'])
                source_id.append(datax['bc_source_id'])
        answer=pd.DataFrame(list(zip(firm_name,source_id)),columns=['bc_firm_name','bc_source_id'])
        return answer

In [227]:
search=["345%20Park%20Avenue","GSO%20Capital","BREP","blackstone.com","blackstone","Blackstone%20Real%20Estate%20Partners"]
data=pd.DataFrame()
for i in search:
    data=pd.concat([data,getdata(i)],axis=0)
data

,bc_firm_name,bc_source_id
0,GSO CAPITAL PARTNERS LP,137519
1,GSO CAPITAL ADVISORS LLC,152294
0,BLACKSTONE MEZZANINE ADVISORS LP,109099
1,BLACKSTONE MANAGEMENT PARTNERS LP,109108
2,BLACKSTONE WEALTH MANAGEMENT,160794
3,BLACKSTONE ENERGY CORPORATION,14098
4,BLACKSTONE ALTERNATIVE ASSET MANAGEMENT LP,107580
5,BLACKSTONE COMMUNICATIONS ADVISORS I LLC,109106
6,BLACKSTONE DEBT ADVISORS L.P.,120934
7,BLACKSTONE REAL ESTATE ADVISORS LP,109110


In [229]:
from PyPDF2 import PdfFileWriter, PdfFileReader
from io import BytesIO
def getauthor(ID=''):
    newurl="https://adviserinfo.sec.gov/IAPD/IAPDFirmSummary.aspx?ORG_PK="+ID
    newpage=urllib.request.urlopen(newurl)
    soup = BeautifulSoup(newpage)
    if str(soup).find("BRCHR_VRSN_ID")==-1:
        return ''
    pdfid=str(soup)[str(soup).find("BRCHR_VRSN_ID"):].split("\"")[0].split("=")[1]
    pdfurl="https://adviserinfo.sec.gov/IAPD/Content/Common/crd_iapd_Brochure.aspx?BRCHR_VRSN_ID="+pdfid
    pdfpage=urllib.request.urlopen(pdfurl)
    memoryFile = BytesIO(pdfpage.read())
    pdfFile = PdfFileReader(memoryFile)
    if '/Author' in pdfFile.documentInfo:
        return pdfFile.getDocumentInfo()['/Author']
    else :
        return ''

In [230]:
authorlist=[]
for i in list(data['bc_source_id']):
    authorlist.append(getauthor(i))
authorlist


The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")
 [__init__.py:181]


['Ascendant Template', 'Ascendant Template', 'Ascendant Template', '', '', 'TBG', '', 'Ascendant Template', '', '', '', 'TBG', '']

In [234]:
set(authorlist)

{'', 'TBG', 'Ascendant Template'}

### As we see the two found authors were Ascendant Template or TBG